# 🎯 HEGEMON Phase 2.4 - Przykład Human-in-the-Loop

Ten notebook pokazuje jak używać naprawionego interfejsu Phase 2.4 do interaktywnego uczestnictwa w debacie agentów.

## Co zostało naprawione:
- ✅ **Session state persistence** - czyste formularze dla każdego checkpointu
- ✅ **Widget responsiveness** - polling mechanism zapobiega zawieszaniu się Jupyter kernela
- ✅ **Error handling** - defensywne sprawdzanie danych wejściowych
- ✅ **Manual control** - przycisk "Auto-Approve" dla szybkiego przejścia dalej

## 📦 Krok 1: Instalacja i Import

Najpierw upewnij się, że masz wszystkie wymagane pakiety:

In [ ]:
# Odkomentuj jeśli potrzebujesz zainstalować ipywidgets
# !pip install ipywidgets

# Zainstaluj wymagane pakiety z projektu
# !pip install -r requirements.txt

In [ ]:
import sys
import logging
from pathlib import Path

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Basic imports ready")

## 🔧 Krok 2: Import HITL Components

Importujemy naprawione komponenty Phase 2.4:

In [ ]:
# Import głównych komponentów HEGEMON
from hegemon.graph_hitl_v3 import create_hegemon_graph_hitl_v3
from hegemon.config.settings import get_settings

# Import modeli HITL
from hegemon.hitl import (
    CheckpointUI,
    InterventionMode,
    FeedbackDecision,
    create_checkpoint_ui,
)

print("✅ HITL components imported")
print(f"   HITL version: {__import__('hegemon.hitl').__version__}")

## ⚙️ Krok 3: Konfiguracja

Ładujemy ustawienia i weryfikujemy konfigurację:

In [ ]:
settings = get_settings()

print("✅ Settings loaded")
print(f"\nAgents Configuration:")
print(f"  Katalizator: {settings.katalizator.model}")
print(f"  Sceptyk: {settings.sceptyk.model}")
print(f"  Gubernator: {settings.gubernator.model}")
print(f"  Syntezator: {settings.syntezator.model}")
print(f"\nDebate Config:")
print(f"  Max cycles: {settings.debate.max_cycles}")
print(f"  Consensus threshold: {settings.debate.consensus_threshold}")

## 🏗️ Krok 4: Stwórz Graf z Checkpointami

Tworzymy graf HEGEMON z włączoną funkcjonalnością Human-in-the-Loop:

In [ ]:
# Stwórz graf z checkpointami
graph = create_hegemon_graph_hitl_v3()

print("✅ HEGEMON graph created with HITL checkpoints")
print(f"   Nodes: {list(graph.nodes.keys())}")

## 📋 Krok 5: Zdefiniuj Misję

Określ zadanie dla agentów:

In [ ]:
# Prosta misja testowa
MISSION = """
Zaprojektuj strategię wdrożenia ML pipeline dla firmy e-commerce.

Wymagania:
- Budżet: 200k PLN
- Timeline: 6 miesięcy
- Team: 3 ML engineers + 1 DevOps
- Cel: Deployment systemu rekomendacyjnego produktów

Dostarcz: Plan wdrożenia z workflow i analizą ryzyka.
"""

print("📋 Mission defined:")
print(MISSION)

## 🎮 Krok 6: Wybierz Tryb Interwencji

Dostępne tryby:
- **`observer`** - Oglądasz debatę bez możliwości ingerencji (checkpointy są pokazywane, ale automatycznie zatwierdzane)
- **`reviewer`** - Możesz zatwierdzać/odrzucać output agentów
- **`collaborator`** - Pełna kontrola z możliwością rewizji i dodawania własnych wskazówek

In [ ]:
# Wybierz tryb (zmień na 'collaborator' dla pełnej kontroli)
intervention_mode = InterventionMode.REVIEWER

# Stwórz stan początkowy
initial_state = {
    "mission": MISSION,
    "contributions": [],
    "cycle_count": 1,
    "current_consensus_score": 0.0,
    "final_plan": None,
    # HITL-specific fields
    "intervention_mode": intervention_mode.value,
    "current_checkpoint": None,
    "human_feedback_history": [],
    "paused_at": None,
    "revision_count_per_checkpoint": {},
    "checkpoint_snapshots": {},
}

print(f"✅ State initialized with mode: {intervention_mode.value}")

## 🚀 Krok 7: Uruchom Debatę z Checkpointami

**WAŻNE:** Po uruchomieniu tej komórki:
1. Debata się rozpocznie
2. W checkpointach pojawią się interaktywne widgety (radio buttons, text areas)
3. Możesz:
   - ✅ **Approve** - zatwierdź output i kontynuuj
   - ✏️ **Request Revision** - poproś agenta o poprawki (musisz podać guidance)
   - ❌ **Reject** - odrzuć i zakończ debatę
   - ⏩ **Auto-Approve** - szybkie zatwierdzenie bez czekania

**Naprawione problemy:**
- Formularze są czyste dla każdego nowego checkpointu
- Widget callbacks działają bez zawieszania kernela
- Można użyć "Auto-Approve" jeśli nie chcesz czekać 10 minut timeout

In [ ]:
print("🎭 Starting HEGEMON debate with Human-in-the-Loop...")
print("⏳ This may take a few minutes...")
print("\n" + "=" * 80)

# Uruchom debatę
final_state = graph.invoke(
    initial_state,
    config={"recursion_limit": 100}
)

print("\n" + "=" * 80)
print("✅ Debate completed!")

## 📊 Krok 8: Przejrzyj Wyniki

Sprawdź co się wydarzyło w debacie:

In [ ]:
print("=" * 80)
print("📊 DEBATE SUMMARY")
print("=" * 80)
print()

print(f"Total cycles: {final_state['cycle_count']}")
print(f"Final consensus: {final_state.get('current_consensus_score', 0.0):.2f}")
print(f"Total contributions: {len(final_state['contributions'])}")
print()

print("=" * 80)
print("👤 HUMAN INTERVENTIONS")
print("=" * 80)
print()

feedback_history = final_state.get('human_feedback_history', [])
print(f"Total feedback submissions: {len(feedback_history)}")
print(f"Checkpoints reached: {len(final_state.get('checkpoint_snapshots', {}))}")
print()

if feedback_history:
    for i, fb in enumerate(feedback_history, 1):
        print(f"{i}. Checkpoint: {fb.get('checkpoint', 'unknown')}")
        print(f"   Decision: {fb.get('decision', 'unknown')}")
        if fb.get('guidance'):
            print(f"   Guidance: {fb['guidance'][:100]}...")
        print()
else:
    print("No human feedback recorded (observer mode or auto-approved)")
    print()

## 📝 Krok 9: Zobacz Finalny Plan

Wyświetl wygenerowany plan strategiczny:

In [ ]:
if final_state.get('final_plan'):
    plan = final_state['final_plan']
    
    print("=" * 80)
    print("🎯 FINAL STRATEGIC PLAN")
    print("=" * 80)
    print()
    
    print("📋 MISSION OVERVIEW:")
    print("-" * 80)
    print(plan.mission_overview)
    print()
    
    print(f"👥 REQUIRED AGENTS ({len(plan.required_agents)}):")
    print("-" * 80)
    for i, agent in enumerate(plan.required_agents, 1):
        print(f"{i}. {agent.role}")
        print(f"   {agent.description}")
        print(f"   Skills: {', '.join(agent.required_skills[:3])}")
        print()
    
    print(f"📊 WORKFLOW ({len(plan.workflow)} steps):")
    print("-" * 80)
    for step in plan.workflow[:5]:
        deps = f" [depends on: {step.dependencies}]" if step.dependencies else ""
        print(f"{step.step_id}. {step.description}{deps}")
    if len(plan.workflow) > 5:
        print(f"... and {len(plan.workflow) - 5} more steps")
    print()
    
    print("⚠️ RISK ANALYSIS:")
    print("-" * 80)
    print(plan.risk_analysis)
    print()
else:
    print("❌ No final plan generated")

## 💾 Krok 10: Zapisz Wyniki

Eksportuj wyniki do plików:

In [ ]:
import json
from datetime import datetime

# Stwórz folder output jeśli nie istnieje
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

# Przygotuj dane do zapisu
output_data = {
    "timestamp": datetime.now().isoformat(),
    "mission": final_state["mission"],
    "intervention_mode": final_state["intervention_mode"],
    "debate_summary": {
        "total_cycles": final_state["cycle_count"],
        "final_consensus_score": final_state.get("current_consensus_score", 0.0),
        "total_contributions": len(final_state["contributions"]),
        "human_interventions": len(final_state.get("human_feedback_history", [])),
    },
    "human_feedback_history": final_state.get("human_feedback_history", []),
    "contributions": [
        {
            "agent_id": c.agent_id,
            "type": c.type,
            "cycle": c.cycle,
            "content": c.content,
            "rationale": c.rationale,
        }
        for c in final_state["contributions"]
    ],
}

if final_state.get("final_plan"):
    output_data["final_plan"] = final_state["final_plan"].model_dump()

# Zapisz do pliku JSON
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = output_dir / f"hegemon_hitl_{timestamp}.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False, default=str)

print(f"✅ Results saved to: {output_file}")
print(f"📊 File size: {output_file.stat().st_size / 1024:.1f} KB")

## 🎉 Gotowe!

### Co dalej?

1. **Eksperymentuj z trybami:**
   - Zmień `intervention_mode` na `InterventionMode.COLLABORATOR` dla pełnej kontroli
   - Użyj `InterventionMode.OBSERVER` żeby tylko obserwować

2. **Testuj guidance:**
   - Przy "Request Revision" dodaj szczegółowe wskazówki
   - Sprawdź jak agent odpowiada na Twoje sugestie

3. **Użyj zaawansowanych opcji:**
   - "Priority Claims" - zaznacz kluczowe twierdzenia
   - "Flagged Concerns" - wskaż wątpliwości dla Sceptyka

### Naprawione Problemy:
- ✅ Session state resetuje się między checkpointami
- ✅ Widget callbacks działają płynnie
- ✅ Dodano przycisk "Auto-Approve"
- ✅ Lepsza obsługa błędów

**Miłej zabawy z HEGEMON Phase 2.4!** 🚀